In [ ]:
import urllib 

import pandas as pd
import numpy as np 

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

## figure out which stations we need

In [ ]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [ ]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys()),
                    dtype={'ID': str, 'LATITUDE':float, 'LONGITUDE':float, 
                           'ELEVATION':float, 'STATE':str, 'NAME':str, 
                           'GSN FLAG': str, 'HCN/CRN FLAG': str, 'WMO ID':str})

In our documentation sheet https://docs.opendata.aws/noaa-ghcn-pds/readme.html
ID = the station identification code.
* The first two characters denote the FIPS country code
So we're going to use that to get all the US Station IDs, but first we're going to check what it returns, which 
is states + DC + PR + VI

In [ ]:
df[df['ID'].str.startswith('US')]['STATE'].unique()

In [ ]:
# So lets get all the US country codes where the STATE is not in ['PR', 'VI'] 'cause it's easier to map
# Also we need a WMO ID 'cause thats how we cross reference against mos
# this is just a check that our filter is correct

In [ ]:
mask = (df['ID'].str.startswith('US') & ~ df['STATE'].isin(['PR', 'VI']) & ~df['WMO ID'].isnull())
df[mask]['STATE'].unique()

In [ ]:
df[mask]

In [ ]:
# we're getting the list of US unique stations
us_stations = df[mask]
len(us_stations['ID'])

In [ ]:
us_stations.head()

### Lets sample that!
Do we really need all our stations?

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.GeoDataFrame(us_stations, geometry=gpd.points_from_xy(us_stations['LONGITUDE'], us_stations['LATITUDE']))

In [ ]:
gdf.plot(figsize=(10,5))

let's find those outliers by looking at the longitude

In [ ]:
gdf.sort_values('LONGITUDE', ascending=False).head()

In [ ]:
# let's drop them:
gdf_w = gdf[gdf['LONGITUDE']<0]
gdf_w.plot()

We really don't need that dense of a network of plots, let's sample by 10 percent and see what happens

In [ ]:
gdf_w.sample(frac=.10).plot(figsize=(10,5))

In [ ]:
#1% which is about 6 stations
gdf_w.sample(frac=.01).plot(figsize=(10,5))

In [ ]:
# try your own 
frac = ?
gdf_w.sample(frac=frac).plot(figsize=(10,5))

In [ ]:
# lets use the 60 stations from our sample to filter down our dataset
us_stations = gdf_w.sample(frac=.1)['ID']

## Filter MOS down to our sample set of stations
OUR MOS codes aren't matching up w/ our WMO above so we're gonna find the nearest neighbors to the above stations
This is a spreadsheet of where the mos stations are 

In [ ]:
mdf = pd.read_csv("https://www.weather.gov/source/mdl/tables/MOS_stationtable_20190702.csv")

In [ ]:
# us stations only have 4 values
mosdf = mdf[mdf['Station'].str.len()==4]

In [ ]:
mgdf = gpd.GeoDataFrame(mosdf,geometry=gpd.points_from_xy(mosdf['Longitude'], mosdf['Latitude']))

In [ ]:
len(mgdf['Station'].unique())

## Let's find our nearest neighbor
https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe/222316

In [ ]:
# compute distance between each station and all the others
def distance(row):
    return mgdf['geometry'].distance(row['geometry'])
    
distances = gdf_w.apply(distance, axis=1)

In [ ]:
distances.shape # rows in gdf_w x rows in mgdf

In [ ]:
# get the id of the closet station by finding the index of the station with the minimum distance
# and then create a new column in our gdf_w with the closest MOS station
stationloc = np.argmin(distances.values,axis=1)

In [ ]:
stationloc.shape, mgdf.shape, gdf_w.shape

In [ ]:
# from mgdf pull the station closest in distance to the gdf station for that row
gdf_w['neighbor'] = mgdf.iloc[stationloc]['Station'].values

In [ ]:
# this is our lookup comparison
gdf_w[['ID', 'neighbor']].head()

In [ ]:
#lets merge in the mgdf since we're going to end up wanting the center lat/lon for plotting
gdmo = gdf_w.merge(mgdf, left_on = 'neighbor', right_on='Station', suffixes=('_ghcn', '_mos'))

In [ ]:
gdmo.head()

In [ ]:
#let's save out what we need:
gdmo[['ID', 'LATITUDE', 'LONGITUDE', 'Station', 'Latitude', 'Longitude']].to_csv("ghcn_mos_lookup.csv")

## Filter GHCN down to our sample set of stations

In [ ]:
# the above lines up w/ mos, let's filter GHCN down to the us_stations
YEAR = 2017
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  
                 names=names, dtype={'DATA_VALUE':'object'})

In [ ]:
dsus = ds[ds['ID'].isin(gdmo['ID'])]

In [ ]:
dsus.head()

In [ ]:
# Lets only get the columns we want
dsus_subset = dsus[['ID', 'DATE', 'ELEMENT', 'DATA_VALUE',  'M-FLAG', 'Q-FLAG']]
dsus_subset.head()

In [ ]:
# let's save out to a single largeish CSV
sample = 100
dsus.compute().to_csv(f"GHCN_{YEAR}.csv")

## Filter MOS down to just stations in our lookup

In [ ]:
mos = dd.read_csv(f'Examples/mos/modelrun/mav{YEAR}*.csv', 
                  dtype={'CLD':object, 'OBV':object, 'TYP':object,
                         'Q06':object, 'Q12':object, 'T06':object, 
                         'T12':object}).compute()

In [ ]:
mos[mos['station'].isin(gdmo['Station'])].to_csv(f"MOS_{YEAR}.csv")